In [2]:
# Create the data frame with the corrected syntax
my_data3 <- data.frame(
    t = c(2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014),
    xt = c(-0.56, 0.07, 0.12, 1.71, 0.46, -1.26, -0.68, -0.44, 1.22, 0.35),
    yt = c(22,13,27,15,10,6,18,9,17,12),
    ht = c(14,13,25,15,17,19,22,20,16,14)
)

# View the data frame
print(my_data3)

      t    xt yt ht
1  2005 -0.56 22 14
2  2006  0.07 13 13
3  2007  0.12 27 25
4  2008  1.71 15 15
5  2009  0.46 10 17
6  2010 -1.26  6 19
7  2011 -0.68 18 22
8  2012 -0.44  9 20
9  2013  1.22 17 16
10 2014  0.35 12 14


In [3]:
# Load necessary library
library(tseries)

# Perform ADF test for each variable
adf_xt <- adf.test(my_data3$xt)
adf_yt <- adf.test(my_data3$yt)
adf_ht <- adf.test(my_data3$ht)

# Print results
print(adf_xt)
print(adf_yt)
print(adf_ht)



	Augmented Dickey-Fuller Test

data:  my_data3$xt
Dickey-Fuller = -3.2817, Lag order = 2, p-value = 0.09421
alternative hypothesis: stationary


	Augmented Dickey-Fuller Test

data:  my_data3$yt
Dickey-Fuller = -1.5935, Lag order = 2, p-value = 0.7272
alternative hypothesis: stationary


	Augmented Dickey-Fuller Test

data:  my_data3$ht
Dickey-Fuller = -0.92042, Lag order = 2, p-value = 0.9323
alternative hypothesis: stationary



In [ ]:
adf_xt <- adf.test(my_data3$xt)
adf_yt <- adf.test(my_data3$yt)
adf_ht <- adf.test(my_data3$ht)

# Print results
print(adf_xt)
print(adf_yt)
print(adf_ht)